In [1]:
import cvxpy as cp
import gurobipy as gp
from gurobipy import GRB
import numpy as np
import names 
import random
from itertools import product

In [2]:
def generate_scenario(num_resources, num_jobs, score_mu, score_sigma, seed, roles=None):
    '''
    Inputs: 
    Number of resources, Number of jobs, 
    mean and standard deviation of random normal scores distribution for resource-job assignments 
    Outputs: 
    random set of resources (first & last name),  
    random set of jobs, 
    job-assignment-scores dictionary (first & last name tuple keys, score values),
    job-assignment-costs dictionary (first & last name tuple keys, cost values),
    '''
    random.seed(seed)
    np.random.seed(seed)
    if roles is None:
        roles = {"Architect", "BackEndEngineer", "FrontEndEngineer",
                 "Tester", "DataScientist", "DataEngineer"}
    # P.D.F. of resource costs follows Benford's law, having support {1,2,...,9}
    # Benford's law states that in listings, tables of statistics, etc., the digit 1 tends to occur with probability ∼30%, 
    # much greater than the expected 11.1% (i.e., one digit out of 9). 
    # Benford's law can be observed, for instance, by examining tables of logarithms and noting that the first pages are much more worn and smudged than later pages. 
    benford = [np.log10((i+1)/i) for i in range(1,10)]
    # Sample resource names
    resources = {names.get_full_name(i) for i in range(num_resources)}
    # Sample job requirements, given that all roles are equally likely to be selected
    # multinomial arguments are numeber of experiments (jobs, here), probability of selecting a job and output shape
    req = np.random.multinomial(num_jobs, [1/len(roles)]*len(roles), size=1)[0] 
    jobs = set()
    # Assign ID to each job position
    for i, role in enumerate(roles):
        jobs = jobs.union(set(map(''.join, zip([role]*req[i], [str(x).zfill(int(np.log10(num_jobs))+1) 
                                                               for x in range(1,req[i]+1)]))))
    scores = {}
    costs = {}
    # Sample matching score and cost for each potential assignment
    for pair in product(resources, jobs):
        scores[pair] = int(np.clip(np.random.normal(score_mu, score_sigma), 0, 100))
        costs[pair] = random.choices(list(range(1,10)), weights=benford, k=1)[0]
    return resources, jobs, scores, costs 

# Get Optimal Solution thru CVXPY implementation in the BAD WAY through dictionary variables!

In [3]:
def optimize(num_resources, num_jobs, score_mu, score_sigma, seed, roles, budget, job_gap_penalty):

    res, job, ms, cst = generate_scenario(num_resources, num_jobs, score_mu, score_sigma, seed, roles) 
    
    X = {}
    for a in ms.keys():
        X[a] = cp.Variable(boolean=True, name="assign")

    # Create a dictionary variable with jobs as keys and corresponding gap variables as values
    # gap binary to indicate not all jobs are necessarily filled due to budget cap;
    # 1 if a job is not filled and 0 otherwise
    G = {}
    for g in job:
        G[g] = cp.Variable(boolean=True, name="gap")

    # Create constraints.
    constraints = []

    # job constraint: for each job exactly one resource must be assigned to the job or
    # the corresponding gap variable must be set to 1:
    for j in job:
        X_r = 0
        for r in res:
            X_r += X[r, j]
        constraints += [
            X_r + G[j] == 1
            ]

    # resource constraint: for each resource, at most one job can be assigned to him/her:
    for r in res:
        X_j = 0
        for j in job:
            X_j += X[r, j]
        constraints += [
            X_j <= 1
        ]

    # budget constraint: cost of assigning resources to fill job requirements not to exceed budget
    constraints += [
        np.array(list(cst.values())) @ np.array(list(X.values())) <= budget,
    ]

    # Form objective.
    obj = cp.Maximize(np.array(list(ms.values())) @ np.array(list(X.values()))
                      - job_gap_penalty * cp.sum(list(G.values())))
    
    # Form and solve problem.
    prob = cp.Problem(obj, constraints)
    prob.solve(solver=cp.GUROBI, verbose=False) # or cp.CBC
    
    print("status:", prob.status)
    print("optimal value", prob.value)
    
    return X, G
    

In [4]:
res_job_assign, job_gap = optimize(
    num_resources=3, 
    num_jobs=3,
    score_mu=50, 
    score_sigma=15, 
    seed=10101, 
    roles=None, 
#     roles = {"Architect", "BackEndEngineer", "FrontEndEngineer",
#              "Tester", "DataScientist", "DataEngineer"}, 
    budget=10, 
    job_gap_penalty=101
        )

Academic license - for non-commercial use only - expires 2021-02-17
Using license file C:\Users\mghan\gurobi.lic
status: optimal
optimal value 169.0


In [5]:
for k in res_job_assign:
    print(k, res_job_assign[k].value)

('Gaye Botner', 'Architect1') 1.0
('Gaye Botner', 'DataScientist1') -0.0
('Gaye Botner', 'BackEndEngineer1') 0.0
('Karen Bartholomew', 'Architect1') -0.0
('Karen Bartholomew', 'DataScientist1') 1.0
('Karen Bartholomew', 'BackEndEngineer1') -0.0
('Garry Boston', 'Architect1') -0.0
('Garry Boston', 'DataScientist1') 0.0
('Garry Boston', 'BackEndEngineer1') 1.0


In [6]:
gap = 0
for k in job_gap:
    gap += job_gap[k].value
    print(k, job_gap[k].value)
print(str(int(gap)) + ' : Number of jobs could have not been assigned due to budget limit out of the total ' + str(len(job_gap)) + '.')

Architect1 -0.0
DataScientist1 -0.0
BackEndEngineer1 -0.0
0 : Number of jobs could have not been assigned due to budget limit out of the total 3.


# Efficient cvxpy implementation through matrix variables

In [16]:
def optimize(num_resources, num_jobs, score_mu, score_sigma, seed, roles, budget, job_gap_penalty):
    '''    
    call a function to generate random resources, jobs, assignmnet scores and costs
    and selects the optmial resource-job combinations to maximize performance score 
    with three contraints of budget, resource assignmnet to at most one job and
    job assigmnet to at most one resource
    
    '''
    res, job, ms, cst = generate_scenario(num_resources, num_jobs, score_mu, score_sigma, seed, roles) 
    
    
    assign_scores = np.array(list(ms.values())).reshape(len(res), len(job))
    assign_cost = np.array(list(cst.values())).reshape(len(res), len(job))
    x = cp.Variable(shape=(len(res), len(job)), boolean=True, name="assign")
    g = cp.Variable(shape=(len(job), ), boolean=True, name="gap")

    constraints = []
    constraints += [cp.sum(x[:, j]) + g[j] == 1 for j in range(len(job))]
    constraints += [cp.sum(x[r, :]) <= 1 for r in range(len(res))]
    constraints += [cp.sum(cp.multiply(assign_cost, x)) <= budget]

    job_gap_penalty=101
    obj = cp.Maximize(cp.sum(cp.multiply(assign_scores, x) - job_gap_penalty * cp.sum(g)))
    prob = cp.Problem(obj, constraints)
    prob.solve(solver=cp.GUROBI, verbose=True)
    
    res_job_assign = ms.copy()
    i = 0
    for key in res_job_assign.keys():
        i += 1
        res_job_assign[key] = x.value.astype(int).reshape(len(res_job_assign), ).tolist()[i - 1]
    
    job_not_filled = dict(zip(job, g.value.astype(int)))
    
    print("status: ", prob.status)
    print("optimal value: ", prob.value)
    print("Number of unfilled jobs: ", int(sum(g.value)))
    
    return x, g, job_not_filled, res_job_assign
    

In [17]:
res_job_assign_bool, job_gap_bool, job_gap, res_job_assign = optimize(
    num_resources=200, 
    num_jobs=200,
    score_mu=50, 
    score_sigma=15, 
    seed=10101, 
    roles=None, 
    budget=200, 
    job_gap_penalty=101
        )

Parameter OutputFlag unchanged
   Value: 1  Min: 0  Max: 1  Default: 1
Changed value of parameter QCPDual to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 401 rows, 40200 columns and 120200 nonzeros
Model fingerprint: 0x2db8da98
Variable types: 0 continuous, 40200 integer (40200 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+00]
  Objective range  [1e+00, 4e+06]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+02]
Found heuristic solution: objective 8.080000e+08
Presolve time: 0.15s
Presolved: 401 rows, 40200 columns, 120200 nonzeros
Variable types: 0 continuous, 40200 integer (40200 binary)

Root relaxation: objective -1.631200e+04, 600 iterations, 0.05 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time


In [10]:
job_gap

{'Tester034': 0,
 'DataScientist006': 0,
 'DataEngineer023': 0,
 'DataEngineer016': 0,
 'BackEndEngineer014': 0,
 'Tester036': 0,
 'DataEngineer025': 0,
 'DataScientist027': 0,
 'DataEngineer010': 0,
 'FrontEndEngineer001': 0,
 'Architect009': 0,
 'BackEndEngineer015': 0,
 'Architect028': 0,
 'DataScientist034': 0,
 'Tester002': 0,
 'BackEndEngineer005': 0,
 'FrontEndEngineer015': 0,
 'BackEndEngineer022': 0,
 'DataScientist007': 0,
 'Architect014': 0,
 'DataEngineer007': 0,
 'Architect017': 0,
 'Tester025': 0,
 'DataScientist031': 0,
 'DataEngineer018': 0,
 'DataScientist012': 0,
 'DataEngineer017': 0,
 'DataScientist021': 0,
 'DataEngineer024': 0,
 'FrontEndEngineer026': 0,
 'FrontEndEngineer024': 0,
 'Architect020': 0,
 'Architect027': 0,
 'FrontEndEngineer012': 0,
 'DataEngineer009': 0,
 'DataEngineer002': 0,
 'BackEndEngineer017': 0,
 'Tester031': 0,
 'BackEndEngineer030': 0,
 'DataScientist032': 0,
 'Tester020': 0,
 'Tester003': 0,
 'Architect025': 0,
 'Architect011': 0,
 'Archit

In [11]:
res_job_assign

{('Janice Rothrock', 'Tester034'): 0,
 ('Janice Rothrock', 'DataScientist006'): 0,
 ('Janice Rothrock', 'DataEngineer023'): 0,
 ('Janice Rothrock', 'DataEngineer016'): 0,
 ('Janice Rothrock', 'BackEndEngineer014'): 0,
 ('Janice Rothrock', 'Tester036'): 0,
 ('Janice Rothrock', 'DataEngineer025'): 0,
 ('Janice Rothrock', 'DataScientist027'): 0,
 ('Janice Rothrock', 'DataEngineer010'): 0,
 ('Janice Rothrock', 'FrontEndEngineer001'): 0,
 ('Janice Rothrock', 'Architect009'): 0,
 ('Janice Rothrock', 'BackEndEngineer015'): 0,
 ('Janice Rothrock', 'Architect028'): 0,
 ('Janice Rothrock', 'DataScientist034'): 0,
 ('Janice Rothrock', 'Tester002'): 0,
 ('Janice Rothrock', 'BackEndEngineer005'): 0,
 ('Janice Rothrock', 'FrontEndEngineer015'): 0,
 ('Janice Rothrock', 'BackEndEngineer022'): 0,
 ('Janice Rothrock', 'DataScientist007'): 0,
 ('Janice Rothrock', 'Architect014'): 0,
 ('Janice Rothrock', 'DataEngineer007'): 0,
 ('Janice Rothrock', 'Architect017'): 0,
 ('Janice Rothrock', 'Tester025'): 0,


# Get Optimal Solution thru Gurobi implementation

In [12]:
def optimize_gurobi(num_resources, num_jobs, score_mu, score_sigma, seed, roles, budget, job_gap_penalty):
    '''    
    call a function to generate random resources, jobs, assignmnet scores and costs
    and selects the optmial resource-job combinations to maximize performance score 
    with three contraints of budget, resource assignmnet to at most one job and
    job assigmnet to at most one resource
    
    '''
    res, job, ms, cst = generate_scenario(num_resources, num_jobs, score_mu, score_sigma, seed, roles) 
    
    
    m = gp.Model("RAP")
    assign = m.addVars(ms.keys(), vtype=GRB.BINARY, name="assign")
    g = m.addVars(job, name="gap")
    m.addConstrs((assign.sum("*", j) + g[j]  == 1 for j in job), name="demand")
    m.addConstrs((assign.sum(r, "*") <= 1 for r in res), name="supply")
    m.addConstr(assign.prod(cst) <= budget, name="Budget")
    job_gap_penalty = 101 # penatly of not filling a job 
    m.setObjective(assign.prod(ms) -job_gap_penalty*g.sum(), GRB.MAXIMIZE)

    return m.optimize()
    

In [13]:
optimized_model = optimize(
    num_resources=200, 
    num_jobs=200,
    score_mu=50, 
    score_sigma=15, 
    seed=10101, 
    roles=None, 
    budget=200, 
    job_gap_penalty=101
        )

Parameter OutputFlag unchanged
   Value: 1  Min: 0  Max: 1  Default: 1
Changed value of parameter QCPDual to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 401 rows, 40200 columns and 120200 nonzeros
Model fingerprint: 0x2db8da98
Variable types: 0 continuous, 40200 integer (40200 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+00]
  Objective range  [1e+00, 4e+06]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+02]
Found heuristic solution: objective 8.080000e+08
Presolve time: 0.14s
Presolved: 401 rows, 40200 columns, 120200 nonzeros
Variable types: 0 continuous, 40200 integer (40200 binary)

Root relaxation: objective -1.631200e+04, 600 iterations, 0.05 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time
